In [1]:
import time
import warnings
import logging
import tensorflow as tf


In [2]:
#tf.function is used to make static computational graph out of dynamic computational graph. this dynamic computational
#graph is in eager execution mode. 

#following are few helper methods

In [3]:
@tf.function
def add(a,b):
    return a+b

@tf.function
def sub(a,b):
    return a-b

@tf.function
def mul(a,b):
    return a*b

@tf.function
def div(a,b):
    return a/b

In [4]:
#when we add @tf.function python converts the function to static graph and than execute
#these functions are executed in graph mode hence speeding up graph execution

print(add(tf.constant(3),tf.constant(4)))

tf.Tensor(7, shape=(), dtype=int32)


In [5]:
print(sub(tf.constant(3),tf.constant(4)))

tf.Tensor(-1, shape=(), dtype=int32)


In [6]:
print(mul(tf.constant(3),tf.constant(4)))

tf.Tensor(12, shape=(), dtype=int32)


In [7]:
print(div(tf.constant(3),tf.constant(4)))

tf.Tensor(0.75, shape=(), dtype=float64)


In [23]:
#we can invoke other tensorflow operations as aswell in tf.function

@tf.function
def matmul(a,b):
    return tf.matmul(a,b)

@tf.function
def linear(m,x,c):
    return add(matmul(m,x),c)

In [22]:
m = tf.constant([[4.0,5.0,6.0]],tf.float32)
m

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[4., 5., 6.]], dtype=float32)>

In [24]:
x = tf.Variable([[100.,100.,100.]],tf.float32)
x

<tf.Variable 'Variable:0' shape=(1, 3) dtype=float32, numpy=array([[100., 100., 100.]], dtype=float32)>

In [25]:
c = tf.constant([[1.0]],tf.float32)
c

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[1.]], dtype=float32)>

In [26]:
linear(m,x,c)


ValueError: in user code:

    <ipython-input-23-d2fe73c8e22d>:9 linear  *
        return add(matmul(m,x),c)
    <ipython-input-8-a4fd70aab974>:5 matmul  *
        return(tf.matmul(a,b))
    C:\Users\Admin\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\util\dispatch.py:201 wrapper  **
        return target(*args, **kwargs)
    C:\Users\Admin\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\ops\math_ops.py:3254 matmul
        a, b, transpose_a=transpose_a, transpose_b=transpose_b, name=name)
    C:\Users\Admin\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\ops\gen_math_ops.py:5642 mat_mul
        name=name)
    C:\Users\Admin\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\op_def_library.py:744 _apply_op_helper
        attrs=attr_protos, op_def=op_def)
    C:\Users\Admin\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\func_graph.py:593 _create_op_internal
        compute_device)
    C:\U

In [ ]:
#running loop in @tf.function
# assign_add() is equal to num = num + x

In [56]:
num = tf.Variable(7) 

In [58]:
@tf.function
def add_times(x):
    for i in tf.range(x):
        num.assign_add(x)     

In [59]:
add_times(5)

In [60]:
print(num)

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=32>


In [61]:
a = tf.Variable(2.0)
b = tf.Variable(3.0)


In [62]:
#function decorated using @tf.function maintain the order of dependencies specified in code
#the below functions first multiply y by b and assigns the value to a than next this updated value of a 
# is then used in b.assign_add(x*a) ie a*x is performed the resulting is added to value of b.
# after this we return updated value of a + updated value of b
@tf.function
def f(x,y):
    a.assign(y*b)
    b.assign_add(x*a)
    return a+b


In [63]:
f(2,4)

<tf.Tensor: shape=(), dtype=float32, numpy=39.0>

In [68]:
# python functions are polymorphic, we can invoke a function with different type of argumetn

a = tf.Variable(9)
@tf.function
def square(x):
    print("INPUT A:",a)
    return a*a


In [69]:
square(a)

INPUT A: <tf.Variable 'Variable:0' shape=() dtype=int32>


<tf.Tensor: shape=(), dtype=int32, numpy=81>

In [72]:
b = tf.Variable(5.0)
b

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=5.0>

In [73]:
square(b)

INPUT A: <tf.Variable 'Variable:0' shape=() dtype=float32>


<tf.Tensor: shape=(), dtype=float32, numpy=25.0>

In [ ]:
#for each input dtype a seperate graph is used for computation

#variables behave differently in eager mode and graph mode

# if we want to see equivalent graphmode of code written in eager mode we use tf.autograph.to_code()